<a href="https://colab.research.google.com/github/mrutherfoord/portfolio/blob/master/SDS_Inventory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SDS Inventory
This is a project where I was able to use Python at my regular job. 

Companies whose employees work with chemicals are required to keep Safety Data Sheets available for anyone to view. These sheets contain important information like first aid measures and toxicological information if a person is exposed to a product. 

In addition to gathering and organizing the SDS sheets, Risk Management informed us that we also needed to provide a spreadsheet that listed each manufacturer and product along with every CAS number, a unique chemical identifier given to every chemical. CAS numbers are comprised of three parts and range between five and ten digits in length. Products without CAS numbers did not need to be included.

Going through each of the roughly 1200 SDS sheets in four departments in order to find and record every chemical would have taken weeks to do manually, so I wrote this program to pull out the CAS numbers from our documents and format them the way Risk Management directed.

All of the SDS sheets were in PDF form, so I first coverted them to excel sheets using an online converter. 

This notebook shows this process for one of the departments.

In [0]:
# import modules
import os
import re
import glob
import pandas as pd
import csv
import numpy as np

In [0]:
def process_excel(folder_path):
    """Consolidating excel files with multiple sheets into single excel files"""

    excel_files = glob.glob(folder_path)
            
    for excel in excel_files:
        sheets_dict = pd.read_excel(excel, sheet_name=None, header=None)
        out = excel.split('.')[0]+'.csv'
        full_table = pd.DataFrame()
    
        for name, sheet in sheets_dict.items():
        # replace NaN values with spaces and make everything lowercase
            full_table = full_table.append(sheet)
        full_table = full_table.fillna('').astype(str).apply(lambda x: x.str.lower())
  
        for col in full_table:
        # replace any non-ASCii characters with space
            full_table[col].replace({r'[^\x00-\x7F]+':' '}, regex=True, inplace=True)
        full_table.reset_index(inplace=True, drop=True)
    
        # export to csv
        full_table.to_csv(out, index=False, encoding='utf-8')                 

In [0]:
# path to convert xlsx to csv 
my_path = 'C:/Users/mturn/Documents/MSDS458/SDS/Light_Shop_SDS_xlsx/*/*.xlsx'

In [0]:
def remove_excel(folder_path):
    """Remove excel files while keeping CSV"""
    for directory, subdir, filename in os.walk(folder_path):
        for file in filename:
            if file.endswith(".xlsx"):
                os.remove(os.path.join(directory, file))

In [0]:
def find_cas_numbers(folder_path):
    """Get manufacturer, product, and CAS numbers"""
    global cas_numbers
    cas_numbers = {}
    global manufacturer
    manufacturer = []
    for directory, subdir, filename in os.walk(folder_path):
        for file in filename:
            file_name = file.split('.')[0]
            sds_file = os.path.join(directory, file)
            dir_name = os.path.basename(os.path.dirname(sds_file))
            manufacturer.append(dir_name)
            with open(sds_file, 'r') as f:
                text = f.read()
                result = set(re.findall('[1-9]{1}[0-9]{1,5}-\d{2}-\d', text))
                cas_numbers.update({file_name:result})
    return

In [0]:
# path to organize csv
my_path = 'C:/Users/mturn/Documents/MSDS458/SDS/Light_Shop_SDS_xlsx/'

In [0]:
remove_excel(my_path)

In [0]:
find_cas_numbers(my_path)

In [0]:
# Make a dataframe from cas_number dictionary
sds_dataframe = pd.DataFrame.from_dict(cas_numbers, orient='index')

In [0]:
sds_dataframe

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
desktop,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Ad-Tech_962_Black_Glue_Stick,557-05-1,24937-78-8,9002-88-4,64742-16-1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Graphite_Powder_Extra_Fine,14808-60-7,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Lock Ease_Graphited_Lock_Fluid,7782-42-5,71-43-2,64742-82-1,91-20-3,2057-10-0,8052-41-3,1305-62-0,506-12-7,57-11-4,100-41-4,...,None,None,None,None,None,None,None,None,None,None
Solder_Paste_Flux,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Circuit_Writer_Conductive_Silver_Ink,7440-22-4,106-65-0,627-93-0,1119-40-0,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
High_Performance_Fluid,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Deco_Art_Dazzling_Metallics_All_Colors,7732-18-5,57-55-6,12001-26-2,28205-96-1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Energizer_Batteries,7782-42-5,7440-66-6,1310-58-3,65997-19-5,1313-13-9,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Goo_Gone,8028-48-6,64742-47-8,5989-27-5,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [0]:
# Combine CAS numbers into one column
sds_dataframe['CAS Numbers'] = sds_dataframe[sds_dataframe.columns[1:]].apply(
    lambda x: ', '.join(x.dropna()), axis=1)

In [0]:
sds_dataframe

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,CAS Numbers
desktop,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,
Ad-Tech_962_Black_Glue_Stick,557-05-1,24937-78-8,9002-88-4,64742-16-1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"24937-78-8, 9002-88-4, 64742-16-1"
Graphite_Powder_Extra_Fine,14808-60-7,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,
Lock Ease_Graphited_Lock_Fluid,7782-42-5,71-43-2,64742-82-1,91-20-3,2057-10-0,8052-41-3,1305-62-0,506-12-7,57-11-4,100-41-4,...,None,None,None,None,None,None,None,None,None,"71-43-2, 64742-82-1, 91-20-3, 2057-10-0, 8052-..."
Solder_Paste_Flux,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,
Circuit_Writer_Conductive_Silver_Ink,7440-22-4,106-65-0,627-93-0,1119-40-0,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"106-65-0, 627-93-0, 1119-40-0"
High_Performance_Fluid,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,
Deco_Art_Dazzling_Metallics_All_Colors,7732-18-5,57-55-6,12001-26-2,28205-96-1,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"57-55-6, 12001-26-2, 28205-96-1"
Energizer_Batteries,7782-42-5,7440-66-6,1310-58-3,65997-19-5,1313-13-9,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"7440-66-6, 1310-58-3, 65997-19-5, 1313-13-9"
Goo_Gone,8028-48-6,64742-47-8,5989-27-5,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,"64742-47-8, 5989-27-5"


In [0]:
# Remove all columns except combined CAS number column
sds_dataframe.drop(sds_dataframe.columns[0:29], axis = 1, inplace = True) 

In [0]:
sds_dataframe

,CAS Numbers
desktop,
Ad-Tech_962_Black_Glue_Stick,"24937-78-8, 9002-88-4, 64742-16-1"
Graphite_Powder_Extra_Fine,
Lock Ease_Graphited_Lock_Fluid,"71-43-2, 64742-82-1, 91-20-3, 2057-10-0, 8052-..."
Solder_Paste_Flux,
Circuit_Writer_Conductive_Silver_Ink,"106-65-0, 627-93-0, 1119-40-0"
High_Performance_Fluid,
Deco_Art_Dazzling_Metallics_All_Colors,"57-55-6, 12001-26-2, 28205-96-1"
Energizer_Batteries,"7440-66-6, 1310-58-3, 65997-19-5, 1313-13-9"
Goo_Gone,"64742-47-8, 5989-27-5"


In [0]:
sds_dataframe['Manufacturer'] = manufacturer
sds_dataframe = sds_dataframe.reset_index()

In [0]:
sds_dataframe.rename(columns={'index':'Product'}, inplace=True)

In [0]:
sds_dataframe = sds_dataframe[['Manufacturer', 'Product', 'CAS Numbers']]

In [0]:
sds_dataframe.replace('_', ' ', regex=True, inplace=True)

In [0]:
# Replace empty cells with NaN values
sds_dataframe['CAS Numbers'].replace('', np.nan, inplace=True)

In [0]:
# Drop rows of products with no CAS numbers
sds_dataframe = sds_dataframe.dropna(axis=0, subset=['CAS Numbers'])

In [0]:
sds_dataframe.insert(loc=2, column="Max Quantity", value=1)

In [0]:
sds_dataframe.insert(loc=4, column="Added", value='X')

In [0]:
sds_dataframe.insert(loc=5, column="Deleted", value='')

In [0]:
# Finished spreadsheet
sds_dataframe.head(10)

,Manufacturer,Product,Max Quantity,CAS Numbers,Added,Deleted
1,Adhesive Technologies Inc,Ad-Tech 962 Black Glue Stick,1,"24937-78-8, 9002-88-4, 64742-16-1",X,
3,AGS Company,Lock Ease Graphited Lock Fluid,1,"71-43-2, 64742-82-1, 91-20-3, 2057-10-0, 8052-...",X,
5,CAIG Laboratories,Circuit Writer Conductive Silver Ink,1,"106-65-0, 627-93-0, 1119-40-0",X,
7,DecoArt,Deco Art Dazzling Metallics All Colors,1,"57-55-6, 12001-26-2, 28205-96-1",X,
8,Energizer Battery Manufacturing,Energizer Batteries,1,"7440-66-6, 1310-58-3, 65997-19-5, 1313-13-9",X,
9,Goo Gone,Goo Gone,1,"64742-47-8, 5989-27-5",X,
10,Henkel Corporation,Loctite Clear Silicone Sealant,1,"17689-77-9, 7631-86-9, 4253-34-3, 64742-46-7, ...",X,
13,Henkel Corporation,Loctite Threadlocker Blue 242,1,"57-55-6, 80-15-9, 98-82-8, 9004-96-0, 2081-07-...",X,
14,HI-Watt Battery Industry Co,Alkaline Button Cell Battery AG13,1,"2015-07-1, 7732-18-5, 9003-53-6, 1310-58-3, 74...",X,
15,High End Systems,Atmospheres HQ Light Enhancement Fluid,1,112-27-6,X,


In [0]:
sds_dataframe.to_excel("WirtzLight_Shop_SDS_Inventory.xlsx", index=False)